In [8]:
import numpy, sys, copy
from datetime import datetime
startTime = datetime.now()
grid = numpy.zeros((0, 0))

ADJACENTS = {(0, -1), (0, 1), (-1, 0), (1, 0)}

class Node():
    def __init__(self, position = None, distance = sys.maxsize):
        self.position = position
        self.distance = distance
        
    def __eq__(self, other):
        return self.position == other.position
    
def pathfind(start, end, unvisited):
    t = datetime.now()
    for node in unvisited:
        if node.position == start:
            node.distance = 0
            break
            
    while unvisited:
        current_node = unvisited[0]
        for node in unvisited[1:]:
            if node.distance < current_node.distance:
                current_node = node

        if current_node.position == end:
            print(datetime.now()-t)
            return current_node.distance

        for adjacent in ADJACENTS:
            x, y = current_node.position[0] + adjacent[0], current_node.position[1] + adjacent[1]

            for node in unvisited:
                if node.position == (x, y) and node.distance > current_node.distance + 1:
                    node.distance = current_node.distance + 1
                    break

        unvisited.remove(current_node)
    
locations = {}
unvisited = []
x, y = 0, 0
with open('input.txt', 'r') as f:
    for line in f.readlines():
        for x, n in enumerate(line.strip()):
            if n != '#':
                unvisited.append(Node((y, x)))    
            if n.isdigit():
                locations[int(n)] = y, x
        y += 1
f.close()

distances = {}
for start_point in locations:
    for end_point in locations:
        if start_point != end_point and tuple(sorted([start_point, end_point])) not in distances.keys():
            new_path = copy.deepcopy(unvisited)
            distances[tuple(sorted([start_point, end_point]))] = pathfind(locations[start_point], locations[end_point], new_path)

from itertools import permutations
routes = permutations(range(1, int(max(locations.keys())) + 1))
min_distance_p1, min_distance_p2 = float('inf'), float('inf')
for route in routes:
    distance = 0
    start_point = 0
   
    for pos in route:
        distance += distances[tuple(sorted([start_point, pos]))]
        start_point = pos
   
    if distance < min_distance_p1:
        min_distance_p1 = distance
        
    distance += distances[tuple(sorted([start_point, 0]))]
    
    if distance < min_distance_p2:
        min_distance_p2 = distance
    
# number of distances = n! / 2*(n-2)!
# number of combinations stemming from 0 = (n-1)!
print(min_distance_p1, min_distance_p2)
print(str(datetime.now() - startTime)[:-3])


ValueError: max() arg is an empty sequence

In [7]:
import numpy, sys, copy
from datetime import datetime
startTime = datetime.now()
grid = numpy.zeros((0, 0))

ADJACENTS = {(0, -1), (0, 1), (-1, 0), (1, 0)}

filepath = 'input.txt'
filepath = 'test.txt'
    
def pathfind(start, end, unvisited, doors, keys):
    t = datetime.now() 
    visited = {}
    #print(tuple(start))
    paths = {tuple(start): ''}
    while unvisited:
        current_node = list(unvisited.keys())[0]
        current_val = unvisited[list(unvisited.keys())[0]]
        for key, val in unvisited.items():
            if val < current_val:
                current_node = key
                current_val = val
        #if current_node == end:
        #    print(datetime.now()-t)
        #    return current_val
        end = True
        for adjacent in ADJACENTS:
            x, y = current_node[0] + adjacent[0], current_node[1] + adjacent[1]

            for key, val in unvisited.items():
                if key == (x, y) and val > current_val + 1:
                    unvisited[key] = current_val + 1
                    
                    if (x, y) in doors.keys():
                        paths[x, y] = paths[current_node] + doors[x, y]
                    elif (x, y) in keys.keys():
                        paths[x, y] = paths[current_node] + keys[x, y]
                    else:
                        paths[x, y] = paths[current_node]
                    
                    #print('DOING')
                    end = False
                    break
                    
        #print(current_node, end)
                    
        if not end:
            del paths[current_node]
            
        visited[current_node] = unvisited.pop(current_node)
        
    #print(str(datetime.now() - startTime)[:-3])
    #for key, path in paths.items():
    #   if path != '':
    #       print(key, path)
    #print(paths)
    locks = set(paths.values())
    #for lock in locks:
    #    print(lock)
        
    sorted_locks = sorted(list(locks))
    
    for lock in sorted_locks[:]:
        for other_lock in sorted_locks[:]:
            if lock != other_lock and lock in other_lock:
                sorted_locks.remove(lock)
                #print('removing', lock)
                break
    
    return sorted_locks
    
doors = {}
keys = {}
unvisited = {}
start = None
y = 0
with open(filepath, 'r') as f:
    for line in f.readlines():
        for x, n in enumerate(line.strip()):
            if n == '@':
                unvisited[x, y] = 0
                start = [x, y]
            elif n == '.':
                unvisited[x, y] = float('inf')
            elif n.isalpha():
                if n.islower():
                    keys[x, y] = n
                elif n.isupper():
                    doors[x, y] = n
                else:
                    print('sad elf')
                unvisited[x, y] = float('inf')
            else:
                pass
        y += 1
        

# get all the possible moves
#    includes  getting keys
#     includes using keys to open doors
#    pathfind from each spot to each spot


#print(unvisited)
#print(doors)
#print(keys)

#print(len(doors.keys()))
#print(len(keys.keys()))

sorted_paths = pathfind(start, None, unvisited, doors, keys)

for n, paths in enumerate(sorted_paths):
    print(str(n).zfill(2), paths)
#print(str(datetime.now() - startTime)[:-3])


00 AbCeEDf
01 aBcd


In [10]:
import itertools
def find_paths(sorted_paths, t, available_keys = []):
    # print('recursion lvel', t, sorted_paths)
    #if t == 5:
    #    return ' '
    options = set()
    for n, path in enumerate(sorted_paths[:]):
        if path[0].islower():
            options.add(path[0])
            #sorted_paths[n] = sorted_paths[n][1:]
        
        if path[0].lower() in available_keys:
            options.add(path[0])
            #sorted_paths[n] = sorted_paths[n][1:]
            
    sorted_paths = [path for path in sorted_paths if path != '']
            
    options = list(options) 
    #available_keys = available_keys + [i for i in options if i.islower()]
    '''
    if len(sorted_paths) != 0 :#and len(options) != 1:
        next_options = find_paths(sorted_paths, available_keys)
        
        options = [i for i in itertools.product(options, next_options)]
        #for option in options:
        #    option = option + '''
    #print('current options', options)
    #print(options)
    #if options == ['o', 'a', 'd', 'n', 'i']:
    #    options = ['a']
        
    
    output_options = set()
    for n, option in enumerate(options):
        new_sorted_paths = sorted_paths[:]
        new_available_keys = available_keys[:]
        
        if option.islower():
            new_available_keys.append(option)
        
        for n in range(len(new_sorted_paths)):
            if option in new_sorted_paths[n]:
                new_sorted_paths[n] = new_sorted_paths[n][1:]
                
        new_sorted_paths = [path for path in new_sorted_paths if path != '']
        #print('currently using option', option)
        #print('new sorted for current option', new_sorted_paths, new_available_keys)
        if len(new_sorted_paths) != 0:
            #print('recursing')
            next_options = find_paths(new_sorted_paths, t+1,  new_available_keys)
            #options[n] = [i for i in itertools.product(option, next_options)]
            #print('next options', next_options)
            for v in next_options:
                output_options.add(option + v)
                #print('possibility', option + v)
        else:
            output_options = (option)
        
    return output_options
    
for i in find_paths(sorted_paths,0):
    print(i)

#print(find_paths(['a', 'abcdefghi', 'abcdefgz'],0))
print('final', len(find_paths(sorted_paths,0)))

aAbBcCedEDf
aAbBcCeEdDf
aAbBcdCeEDf
aAbBcCdeEDf
final 4


In [11]:
import itertools
def find_paths(sorted_paths, t, available_keys = [], current_route = ''):
    #print('recursion lvel', t, sorted_paths)
    options = set()
    for n, path in enumerate(sorted_paths[:]):
        if path[0].islower():
            options.add(path[0])
            #sorted_paths[n] = sorted_paths[n][1:]
        
        if path[0].lower() in available_keys:
            options.add(path[0])
            #sorted_paths[n] = sorted_paths[n][1:]
            
    sorted_paths = [path for path in sorted_paths if path != '']
            
    options = list(options) 
    #available_keys = available_keys + [i for i in options if i.islower()]
    '''
    if len(sorted_paths) != 0 :#and len(options) != 1:
        next_options = find_paths(sorted_paths, available_keys)
        
        options = [i for i in itertools.product(options, next_options)]
        #for option in options:
        #    option = option + '''
    #print('current options', options)
    if options == ['o', 'a', 'd', 'n', 'i']:
        options = ['a']
        
    output_options = set()
    for n, option in enumerate(options):
        new_sorted_paths = sorted_paths[:]
        new_available_keys = available_keys[:]
        
        if option.islower():
            new_available_keys.append(option)
        
        for n in range(len(new_sorted_paths)):
            if option in new_sorted_paths[n]:
                new_sorted_paths[n] = new_sorted_paths[n][1:]
                
        
                
        new_sorted_paths = [path for path in new_sorted_paths if path != '']
        
        #print('currently using option', option)
        #print('new sorted for current option', new_sorted_paths, new_available_keys)
        if bool(len(new_sorted_paths)):
            #print('recursing')
            current_route = current_route + option
            yield from find_paths(new_sorted_paths, t+1,  new_available_keys, current_route)
            #options[n] = [i for i in itertools.product(option, next_options)]
            #print('next options', next_options)
            #for v in next_options:
            #    output_options.add(option + v)
            #    #print('possibility', option + v)
        else:
            yield current_route + option
            #output_options = (option)
        
    #return output_options
    
    

#print(find_paths(['a', 'abcdefghi', 'abcdefgz'],0))
#print(find_paths(sorted_paths,0))

find = find_paths(sorted_paths,0)
print(next(find))
print(next(find))
print(next(find))
print(next(find))

'''
{'aAbBcdCeEDf', 'aAbBcCeEdDf', 'aAbBcCdeEDf', 'aAbBcCedEDf'}
aAbBcdCeEDf

aAbBcCeEdDf

aAbBcCdeEDf

aAbBcCedEDf
'''

dDaAzonuOiINkKpgGjJZqmUbBMxelQLEsPSXyYhcHCrRvVwWtfFT
dDaAzonuOiINkKpgGjJZqmUbBMxelQLEsPSXyYhcHCrRvVwWtfFTF
dDaAzonuOiINkKpgGjJZqmUbBMxelQLEsPSXyYhcHCrRvVwWtfTfF
dDaAzonuOiINkKpgGjJZqmUbBMxelQLEsPSXyYhcHCrRvVwWtfFtT


"\n{'aAbBcdCeEDf', 'aAbBcCeEdDf', 'aAbBcCdeEDf', 'aAbBcCedEDf'}\naAbBcdCeEDf\n\naAbBcCeEdDf\n\naAbBcCdeEDf\n\naAbBcCedEDf\n"

In [225]:
a = [1,2,3,4]
for i in a:
    if i == 1:
        print('check')
        i += 1
        
print(a)

check
[1, 2, 3, 4]


In [139]:
[i for i in itertools.product(['A'], ['b'])]

[('A', 'b')]

In [222]:
def recursive_generator(i = 5):
    recursive_generator(i - 1)
    print(i)
    yield i - 1
    #yield from recursive_generator(i-1)
        
a = recursive_generator()
print(next(a))
print(next(a))

5
4


StopIteration: 

In [5]:
print(bool(4))

True
